In [1]:
import numpy as np
from scipy import optimize

In [2]:
# decide which position is which
n_people = 5
n_positions = 10

# preferences?
p = np.full((n_people,n_positions), fill_value=None)

# insert names
names = ['Anže', 'Devis', 'Matej A.', 'Matej B.', 'Matic']

p[0] = [3,7,0,6,2,1,4,9,8,5]
p[1] = [2,3,1,0,6,7,4,5,8,9]
p[2] = [2,3,7,6,8,9,1,0,4,5]
p[3] = [4,3,5,9,8,7,6,2,1,0]
p[4] = [1,2,3,7,8,6,4,5,0,9]

print(p)

[[3 7 0 6 2 1 4 9 8 5]
 [2 3 1 0 6 7 4 5 8 9]
 [2 3 7 6 8 9 1 0 4 5]
 [4 3 5 9 8 7 6 2 1 0]
 [1 2 3 7 8 6 4 5 0 9]]


In [3]:
def cost(res, args):
    cost = 0
    p, n_people = args
    for person in range(n_people):
        if None in p[person]:
            continue
        unique, counts = np.unique([int(x) for x in res], return_counts=True)
        if np.any(counts > 1):
            return 1000
        cost += np.where(p[person] == int(res[person]))[0][0]**2
    return cost

class MyTakeStep(object):
    def __init__(self, n_people, n_seats):
        self.n_people = n_people
        self.n_seats = n_seats
    def __call__(self,x):
        return np.random.choice(self.n_seats, self.n_people, replace=False)
    
class MyBounds(object):
    def __init__(self, xmax, xmin):
        self.xmax = np.array(xmax)
        self.xmin = np.array(xmin)
    def __call__(self, **kwargs):
        x = kwargs["x_new"]
        tmax = bool(np.all(x <= self.xmax))
        tmin = bool(np.all(x >= self.xmin))
        return tmax and tmin

In [4]:
mystep = MyTakeStep(n_people, n_positions)
mybounds = MyBounds(np.full((n_people),n_positions-1), np.full((n_people),0))
minimizer_kwargs = {"method": "Nelder-Mead", "args": [p, n_people]}

results = []
n_repeat = 100

for i in range(n_repeat):
    x0 = np.random.choice(n_positions, n_people, replace=False)
    res = optimize.basinhopping(cost, x0, minimizer_kwargs = minimizer_kwargs,disp=False, accept_test=mybounds,
                                take_step=mystep
                               )
    int_res = [int(x) for x in res.x]
    cost_res = res.fun
    
    results += [[int_res, np.full((len(int_res)), cost_res)]]
    res_array = np.array(results)
    seating = np.array(res_array[np.argsort(res_array[:,1,0]),0], dtype=int)[0]
    cost_function = np.array(res_array[np.argsort(res_array[:,1,0]),1,0], dtype=int)[0]
                       
    print('Seating', seating, ', ',
          'Cost function', cost_function)

Seating [3 1 2 5 7] ,  Cost function 17
Seating [7 2 3 4 1] ,  Cost function 2
Seating [7 2 3 4 1] ,  Cost function 2
Seating [7 2 3 4 1] ,  Cost function 2
Seating [7 2 3 4 1] ,  Cost function 2
Seating [7 2 3 4 1] ,  Cost function 2
Seating [7 2 3 4 1] ,  Cost function 2
Seating [7 2 3 4 1] ,  Cost function 2
Seating [7 2 3 4 1] ,  Cost function 2
Seating [7 2 3 4 1] ,  Cost function 2
Seating [7 2 3 4 1] ,  Cost function 2
Seating [7 2 3 4 1] ,  Cost function 2
Seating [7 2 3 4 1] ,  Cost function 2
Seating [7 2 3 4 1] ,  Cost function 2
Seating [7 2 3 4 1] ,  Cost function 2
Seating [7 2 3 4 1] ,  Cost function 2
Seating [7 2 3 4 1] ,  Cost function 2


KeyboardInterrupt: 

In [5]:
for person in range(n_people):
    print(names[person], 'seats at position:', seating[person])

Anže seats at position: 7
Devis seats at position: 2
Matej A. seats at position: 3
Matej B. seats at position: 4
Matic seats at position: 1


In [15]:
for person in range(n_people):
    print(names[person], 'seats at position:', seating[person])

Anže seats at position: 3
Devis seats at position: 2
Matej A. seats at position: 7
Matej B. seats at position: 4
Matic seats at position: 1


In [16]:
p

array([[3, 7, 0, 6, 2, 1, 4, 9, 8, 5],
       [2, 3, 1, 0, 6, 7, 4, 5, 8, 9],
       [2, 3, 7, 6, 8, 9, 1, 0, 4, 5],
       [4, 3, 5, 9, 8, 7, 6, 2, 1, 0],
       [1, 2, 3, 7, 8, 6, 4, 5, 0, 9]], dtype=object)

In [7]:
cost([3,2,7,4,1], [p,n_people])

4

In [8]:
cost([7,2,3,4,1], [p,n_people])

2